In [ ]:
import pandas as pd
import os
import requests
import numpy as np
import time
import string
import pprint as pp


path = '/Users/beng/code/spotify/datasets holding'
extension = '.json'
name = 'beng'  # input('What is your name?\n')
files = []
for file in os.listdir(path):
    if file.endswith(extension) and 'Audio' in file or file.endswith(extension) and 'audiobook' in file:
        files.append(file)


df_list = []
for file in files:
    df = pd.read_json(os.path.join(path, file))
    df_list.append(df)

total = 0
for data in df_list:
    total = total + len(data)
# print(f'Merged dataset should have {total} rows')

df_mega = pd.concat(df_list, ignore_index=True)



# CLEANING

# filter out rows with no listen time
df_mega = df_mega[df_mega['ms_played'] != 0]
# transform ms to seconds
df_mega['seconds_played'] = df_mega['ms_played'] / 1000
# transform seconds to minutes
df_mega['minutes_played'] = round(df_mega['seconds_played'] / 60, 2)
# rename columns
df_mega = df_mega.rename(columns={'ts': 'datetime'})
df_mega = df_mega.rename(columns={'conn_country': 'country'})
df_mega = df_mega.rename(columns={'master_metadata_track_name': 'track_name'})
df_mega = df_mega.rename(columns={'master_metadata_album_artist_name': 'artist_name'})
df_mega = df_mega.rename(columns={'master_metadata_album_album_name': 'album_name'})
# cast datetime to datetime
df_mega['datetime'] = pd.to_datetime(df_mega['datetime'])



# add categories for music, audio and audiobook

def categorise(row):
    if pd.isnull(row['track_name']):
        if pd.isnull(row['episode_show_name']):
            return 'audiobook'
        else:
            return 'podcast'
    else:
        if pd.isnull(row['episode_show_name']):
            return 'music'
        else:
            return row['no category']


df_mega['category'] = df_mega.apply(categorise, axis=1)

# drop unecessary columns
df_mega = df_mega.drop(columns=['offline','offline_timestamp','incognito_mode','endTime','audiobookName','chapterName',
                                'authorName','msPlayed', "platform", "ip_addr"])
# drop nulls
df_mega = df_mega[~df_mega[['track_name', 'episode_name', 'audiobook_title']].isnull().all(axis=1)]

df_mega


Merged dataset should have 55789 rows


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 26 11:52:13 2025

@author: J
"""

#%% import libraries

# directory
import os


# file reading
import json

# data analysis
import pandas as pd


#%% read files

os.chdir("DEFINE PATH")

# df to read files into
ds = []

# Iterate over files in directory
for name in os.listdir():
    if "json" and "Audio" in name:
        with open(name, 'r', encoding="utf8") as file:
            ds += json.load(file)
            
# unwrap list of dictionaries into dataframe
df = pd.DataFrame(ds)
#%% clean and split by music and podcasts

# overview
df.info()

# remove audiobook columns (all null)
df = df.loc[:, ["audiobook" not in name for name in df.columns]]

# columns not used
df = df.drop(["platform", "ip_addr", "offline", "offline_timestamp", "incognito_mode"], axis = 1)

# convert time stamp to datetime
df["ts"] = pd.to_datetime(df["ts"], format='%Y-%m-%dT%H:%M:%SZ')

# convert ms_played to min_played
df = df.rename(columns = {"ms_played":"min_played"})
df["min_played"] = df.min_played.map(lambda x: x/60000)

## split music and podcasts and remove irrelevant columns, rename columns
# podcasts
df_pod = df[df["master_metadata_track_name"].isna()]
df_pod = df_pod.loc[:, ["metadata" not in name for name in df.columns]] # song associated columns
df_pod = df_pod.drop("spotify_track_uri", axis = 1) # song associated column
df_pod = df_pod.rename(columns = {"conn_country" : "country_played"})


# music
df_mu = df[df["episode_show_name"].isna()]
df_mu = df_mu.loc[:, ["episode" not in name for name in df.columns]]
df_mu = df_mu.rename(columns = {"conn_country": "country_played",
                                "master_metadata_track_name" : "track_name",
                                "master_metadata_album_artist_name": "artist_name",
                                "master_metadata_album_album_name" : "album_name"})

# check dataframes
# df_pod.info()
# df_mu.info() 

#%% remove artists with fewer than 10 plays

min_filter = df_mu.artist_name.value_counts().sort_values()
min_filter = min_filter[min_filter >= 10]
min_filter = list(min_filter.index)
df_mu = df_mu[df_mu["artist_name"].isin(min_filter)]

ModuleNotFoundError: No module named 'pandas'